In [1]:
import time 
import datetime
from subprocess import PIPE, run
import cantera as ct
import os
import subprocess
import numpy as np
import pandas as pd
import re
import time

In [2]:
current_env = os.environ.copy()
folder = '/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/'

Functions

In [3]:
def clean_slate(): 
    "restore database to how it was (no edits to main's kinetics database)"
    process = subprocess.run(['git', 'reset', '--hard'], stdout=PIPE, stderr=PIPE, cwd='/home/khalil.nor/Code/RMG-database')
    print('Restoring database to main.')
    print(process.stdout)
    print(process.stderr)
    
def checkout_family(family): 
    """
    Checkout the retrained family from commit 
    908a25d5f354e34ba46d44c1bd47b76cad14b5fc from RMG-database branch PFAS_kinetics_rebased_Jan2025.
    """
    #git_commit = '908a25d5f354e34ba46d44c1bd47b76cad14b5fc'
    git_commit = '4fee96d55dc156935f56a4cd4db68c7ab36cfe85' #with updated thermo for CS resonance structures
    family_folder_path = f'input/kinetics/families/{family}/*'
    process = subprocess.run(['git', 'checkout', git_commit, family_folder_path], stdout=PIPE, stderr=PIPE, cwd='/home/khalil.nor/Code/RMG-database')
    print(f'Checking out the following family: {family}.')
    print(process.stdout) 
    print(process.stderr)
    
def checkout_family_from_before_rebase(family): 
    """
    Checkout the retrained family from commit 
    2d5ae5a4fe49390214d156d40839a5895aeccd48from RMG-database branch USNCM_blends.
    """
    git_commit = '2d5ae5a4fe49390214d156d40839a5895aeccd48' #this is the dropped commit from USNCM blends, before Matt's average fix was applied
    family_folder_path = f'input/kinetics/families/{family}/*'
    process = subprocess.run(['git', 'checkout', git_commit, family_folder_path], stdout=PIPE, stderr=PIPE, cwd='/home/khalil.nor/Code/RMG-database')
    print(f'Checking out the following family: {family}.')
    print(process.stdout) 
    print(process.stderr)    
    
    
def checkout_multiple_families_from_before_rebase(families): 
    """
    Checkout the retrained family from commit 
    2d5ae5a4fe49390214d156d40839a5895aeccd48from RMG-database branch USNCM_blends.
    """
    git_commit = '2d5ae5a4fe49390214d156d40839a5895aeccd48'
    for family in families: 
        family_folder_path = f'./input/kinetics/families/{family}/*'
        process = subprocess.run(['git', 'checkout', git_commit, family_folder_path], stdout=PIPE, stderr=PIPE, cwd='/home/khalil.nor/Code/RMG-database')
    print(f'Checking out the following families: {families}.')
    print(process.stdout) 
    print(process.stderr)    

    
    
def checkout_multiple_families(families): 
    """
    Checkout the retrained family from commit 
    908a25d5f354e34ba46d44c1bd47b76cad14b5fc from RMG-database branch PFAS_kinetics_rebased_Jan2025.
    """
    #git_commit = '908a25d5f354e34ba46d44c1bd47b76cad14b5fc'
    git_commit = '4fee96d55dc156935f56a4cd4db68c7ab36cfe85'
    for family in families: 
        family_folder_path = f'./input/kinetics/families/{family}/*'
        process = subprocess.run(['git', 'checkout', git_commit, family_folder_path], stdout=PIPE, stderr=PIPE, cwd='/home/khalil.nor/Code/RMG-database')
    print(f'Checking out the following families: {families}.')
    print(process.stdout) 
    print(process.stderr)

def generate_RMG_model(description, input_type): 
    """
    Generates a new RMG model in a new folder. 
    """
    new_folder = f'./{description}_rebased'
    RMG_model_directory = f'/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/{description}_rebased/'
    if not os.path.exists(RMG_model_directory):
        os.mkdir(RMG_model_directory)
    
    #select the input.py needed 
    input_dict = {'no_PFAS_fams_or_new_libs': 'input_no_PFAS_families_or_new_libs.py', #does not use new PFAS fams or new CH2F2/PFAS libs
                  'no_PFAS_fams_or_PFAS_libs': 'input_no_PFAS_families_or_PFAS_libs.py', #does not use new PFAS fams or new PFAS libs (does use new CH2F2 lib)
                  'no_PFAS_fams': 'input_no_PFAS_families.py', #does not use new PFAS fams, but includes new CH2F2/PFAS libraries
                  'PFAS_fams': 'input.py', 
                  
                }
    input_file = input_dict[input_type]
    
    #copy the necessary files to the folder
    process = subprocess.run(['scp', f'./{input_file}', './run.sh', RMG_model_directory], stdout=PIPE, stderr=PIPE, encoding='utf-8', cwd=folder)
    print(process.stdout)
    print(process.stderr)
    
    #rename the input file so it matches what's in the run.sh
    process = subprocess.run(['mv', input_file, 'input.py'], stdout=PIPE, stderr=PIPE, encoding='utf-8', cwd=RMG_model_directory)
    print(process.stdout)   
    print(process.stderr)
    
    
    #run the RMG job
    process = subprocess.run(['sbatch', 'run.sh'], stdout=PIPE, stderr=PIPE, env=current_env, encoding='utf-8', cwd = RMG_model_directory)
    print(process.stdout)
    print(process.stderr)
    

def convert_to_cti(description, chemkin_indices, format_desc=True): 
    #copy the necessary files to the folder
    
    if format_desc==False: 
        new_folder = f'./{description}/chemkin'
        RMG_model_directory = f'/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/{description}/'

    else: 
        new_folder = f'./{description}_rebased/chemkin'
        RMG_model_directory = f'/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/{description}_rebased/'

    # if not os.path.exists(f'{new_folder}/chemkin/converter.py'):
    process = subprocess.run(['scp', './converter.py', new_folder], stdout=PIPE, stderr=PIPE, cwd=folder)
    print(process.stdout)
    print(process.stderr)

    #run the converter.py script with the specific chemkin indices
    print(RMG_model_directory)
    process = subprocess.run(['python', 'converter.py', str(chemkin_indices)], stdout=PIPE, stderr=PIPE, cwd=f'{RMG_model_directory}chemkin/')
    print(process.stdout)
    print(process.stderr)
    print('finished')
    

def test_flamespeed(description, chemkin_indices, loglevel, format_desc=True, save=True): 
    
    if format_desc==False: 
        new_folder = f'./{description}/chemkin'
        RMG_model_directory = f'/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/{description}/'

    else: 
        new_folder = f'./{description}_rebased/chemkin'
        RMG_model_directory = f'/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/{description}_rebased/'

    copies_folder = f'{RMG_model_directory}chemkin/copies'
    
    beginning_chemkin = chemkin_indices[0]
    ending_chemkin = chemkin_indices[-1]
    
    if len(chemkin_indices)==1:
        list_=chemkin_indices
    else: 
        list_ = list(range(beginning_chemkin, ending_chemkin))

    ctis = []
    for chemkin_index in list_: 
        if chemkin_index<10:
            cti = f'copy_chem000{chemkin_index}.cti'
        if 100>chemkin_index>=10:
            cti = f'copy_chem00{chemkin_index}.cti'        
        if chemkin_index>=100:
            cti = f'copy_chem0{chemkin_index}.cti'
        ctis.append(cti)

    for cti in ctis: 
        
        cti_path = f'{copies_folder}/{cti}'

        print(f'*******************Starting cti: {cti} of {description} ***************')
        gas = ct.Solution(cti_path)
        halocarbon = 'CH3F(1)'

        To = 298
        Po = 1e5 # ct.one_atm


        mole_frac_list = list(np.linspace(0.025, 0.25, 50))
        mole_frac_list=[0.125]

        results = {}

        for i in range(len(mole_frac_list)): 
            try: 
                x = mole_frac_list[i]
                string = f'****************************starting new volume fraction: {x} **************************'
                print(string)

                norm_ox = (1-x)*.21
                mole_frac_dict = {halocarbon: x, 'O2(2)':((1-x)*.21), 'N2':((1-x)*0.79)} 
                #print(f'Unnormalized composition dictionary: {mole_frac_dict}')

                #normalize it to O2 
                mole_frac_dict = {halocarbon: (x/norm_ox), 'O2(2)':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox } 
                #print(f'Normalizing: {mole_frac_dict}')

                gas.TPX = To, Po, mole_frac_dict
                width = 0.08
                flame = ct.FreeFlame(gas, width=width)
                flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1) 
                flame.max_time_step_count = 3000
                flame.solve(loglevel=loglevel, auto=False)
                Su = flame.velocity[0]
                results[x] = Su
                if save==True: 
                    sltn = flame.to_solution_array()
                    pd = sltn.to_pandas()
                    pd.to_csv(f'data/{x}.csv', index=False)

            except Exception as e: 
                print(f'********************passed volume fraction:{mole_frac_list[i]}, error: {e}*************************************')
                pass

        vol_fracs = list(results.keys())
        flame_speeds = list(results.values())

        print(f"flamespeed: {flame_speeds}")




In [ ]:
combined_families = ['CO_CF_bond_dissociation',
                 'PFAS_Hydrolysis', 
                 'Lactone_Formation',
                 'CO2_Elimination_From_Carboxylic_Acid',
                 'Perfluoroalkene_Formation',
                 'Enol_Ether_Formation',
    
                 '1+2_Cycloaddition',
                 'XY_Addition_MultipleBond',
                 'R_Addition_MultipleBond',
                 '1,2_Insertion_CO', 
                 '1,3_Insertion_CO2',
                 '1,3_sigmatropic_rearrangement',
                 'R_Addition_COm',
                 'F_Abstraction', 
]

halocarbon_fams = ['1+2_Cycloaddition',
                 'XY_Addition_MultipleBond',
                 'R_Addition_MultipleBond',
                 '1,2_Insertion_CO', 
                 '1,3_Insertion_CO2',
                 '1,3_sigmatropic_rearrangement',
                 'R_Addition_COm',
                 'F_Abstraction', 
                   
                   
                 '1,2_Insertion_carbene', #these were ones that David's reactions were in (but not PFAS)
                 'CO_Disproportionation',
                 'Disproportionation', 
                 'H_Abstraction',
                 'R_Recombination',
                 'Singlet_Carbene_Intra_Disproportionation',  
                  
                #other git diffs I found
                 '1,2-Birad_to_alkene',
                 '1,2_XY_interchange',
                 'Birad_R_Recombination',
                 'Br_Abstraction',
                   'Cl_Abstraction',
                   'Concerted_Intra_Diels_alder_monocyclic_1,2_shiftH',
                   'Disproportionation-Y',
                   'Disproportionation',
                   'Intra_RH_Add_Endocyclic',
                   'Intra_ene_reaction',
                   'XY_elimination_hydroxyl',
                   'halocarbene_CO_dimerization',
                   'halocarbene_recombination',
                   'intra_H_migration',
                   'intra_halogen_migration',
]

# Making the RMG models

In [ ]:
for fam in halocarbon_edits_that_I_rebased_on_main:
    print(f'Deleting useless folders for {fam}.')
    
    process = subprocess.run(['rm', '-rf', f'./{fam}_rebased/rms/',  f'./{fam}_rebased/pdep/',  f'./{fam}_rebased/species', f'./{fam}_rebased/rms/solver/'], stdout=PIPE, stderr=PIPE)


In [ ]:
#to get all the dropped jobs
input_type = 'no_PFAS_fams_or_new_libs'

for fam in halocarbon_fams:
    
    if not os.path.exists(f'./{fam}_rebased/chemkin'):
        print(f'############## Starting family: {fam} #################')

        #reset the database
        clean_slate()

        #checkout out the new training data
        checkout_family(fam)

        #generate the RMG model
        generate_RMG_model(fam, input_type)

In [ ]:
#to make the projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/all_families_on_rebase_edited_by_Nora
#to get all the dropped jobs
#input_type = 'no_PFAS_fams_or_new_libs'

halocarbon_edits_that_I_rebased_on_main = ['1+2_Cycloaddition',
                 'XY_Addition_MultipleBond',
                 'R_Addition_MultipleBond',
                 '1,2_Insertion_CO', 
                 '1,3_Insertion_CO2',
                 '1,3_sigmatropic_rearrangement',
                 'R_Addition_COm',
                 'F_Abstraction'
                                
                 '1,2_Insertion_carbene', #these were ones that David's reactions were in (but not PFAS)
                 'CO_Disproportionation',
                 'Disproportionation', 
                 'H_Abstraction',
                 'R_Recombination',
                 'Singlet_Carbene_Intra_Disproportionation',  
                  
                #other git diffs I found
                 '1,2-Birad_to_alkene',
                 '1,2_XY_interchange',
                 'Birad_R_Recombination',
                 'Br_Abstraction',
                   'Cl_Abstraction',
                   'Concerted_Intra_Diels_alder_monocyclic_1,2_shiftH',
                   'Disproportionation-Y',
                   'Disproportionation',
                   'Intra_RH_Add_Endocyclic',
                   'Intra_ene_reaction',
                   'XY_elimination_hydroxyl',
                   'halocarbene_CO_dimerization',
                   'halocarbene_recombination',
                   'intra_H_migration',
                   'intra_halogen_migration'
    ] #no NEW PFAS families, these are just the existing families that I updated/Davids updated that weren't on main


print(f'############## Starting #################')

#reset the database
clean_slate()

In [ ]:
#checkout out the new training data
checkout_multiple_families(halocarbon_edits_that_I_rebased_on_main)

#manually submitting sbatch for model

In [ ]:
#to make the projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/all_families_changed_by_Nora_and_new_PFAS/input.py
#ran this cell and the one directly below
input_type = 'no_PFAS_fams_or_new_libs'

all_families_ive_ever_touched = ['CO_CF_bond_dissociation',
                 'PFAS_Hydrolysis', 
                 'Lactone_Formation',
                 'CO2_Elimination_From_Carboxylic_Acid',
                 'Perfluoroalkene_Formation',
                 'Enol_Ether_Formation',
    
                '1+2_Cycloaddition',
                 'XY_Addition_MultipleBond',
                 'R_Addition_MultipleBond',
                 '1,2_Insertion_CO', 
                 '1,3_Insertion_CO2',
                 '1,3_sigmatropic_rearrangement',
                 'R_Addition_COm',
                 'F_Abstraction'
                                
                 '1,2_Insertion_carbene', #these were ones that David's reactions were in (but not PFAS)
                 'CO_Disproportionation',
                 'Disproportionation', 
                 'H_Abstraction',
                 'R_Recombination',
                 'Singlet_Carbene_Intra_Disproportionation',  
                  
                #other git diffs I found
                 '1,2-Birad_to_alkene',
                 '1,2_XY_interchange',
                 'Birad_R_Recombination',
                 'Br_Abstraction',
                   'Cl_Abstraction',
                   'Concerted_Intra_Diels_alder_monocyclic_1,2_shiftH',
                   'Disproportionation-Y',
                   'Disproportionation',
                   'Intra_RH_Add_Endocyclic',
                   'Intra_ene_reaction',
                   'XY_elimination_hydroxyl',
                   'halocarbene_CO_dimerization',
                   'halocarbene_recombination',
                   'intra_H_migration',
                   'intra_halogen_migration'
    ] #no NEW PFAS families, these are just the existing families that I updated/Davids updated that weren't on main


print(f'############## Starting #################')

#reset the database
clean_slate()

In [ ]:
#ran immediately after cell above. 
checkout_multiple_families(all_families_ive_ever_touched)

#manually submitting sbatch for model

In [ ]:
#generating an RMG model for just the families that I have touched 

combined_families = ['CO_CF_bond_dissociation',
                 'PFAS_Hydrolysis', 
                 'Lactone_Formation',
                 'CO2_Elimination_From_Carboxylic_Acid',
                 'Perfluoroalkene_Formation',
                 'Enol_Ether_Formation',
    
                 '1+2_Cycloaddition',
                 'XY_Addition_MultipleBond',
                 'R_Addition_MultipleBond',
                 '1,2_Insertion_CO', 
                 '1,3_Insertion_CO2',
                 '1,3_sigmatropic_rearrangement',
                 'R_Addition_COm',
                 'F_Abstraction', 
]

print(f'############## Starting #################')

#reset the database
clean_slate()

In [ ]:
#ran immediately after cell above. 
checkout_multiple_families_from_before_rebase(combined_families)

#manually submitting sbatch for model

In [ ]:
#rebased_RMG-Py-database/regenerating_model_at_different_commits/all_families_on_rebase_edited_by_Nora/MODEL_with_newCH2F2_PFAS_libs/all_intended_families_included
clean_slate()

In [ ]:
print(families_ive_rebased)

In [ ]:
checkout_multiple_families(families_ive_rebased)

In [ ]:
#making jobs for regenerating_model_at_different_commits/all_families_on_rebase_edited_by_Nora/one_kinfam_at_a_time/
input_type = 'PFAS_fams'

families_ive_rebased = ['CO_CF_bond_dissociation',
                 'PFAS_Hydrolysis', 
                 'Lactone_Formation',
                 'CO2_Elimination_From_Carboxylic_Acid',
                 'Perfluoroalkene_Formation',
                 'Enol_Ether_Formation',
    
                '1+2_Cycloaddition',
                 'XY_Addition_MultipleBond',
                 'R_Addition_MultipleBond',
                 '1,2_Insertion_CO', 
                 '1,3_Insertion_CO2',
                 '1,3_sigmatropic_rearrangement',
                 'R_Addition_COm',
                 'F_Abstraction',
                                
                 '1,2_Insertion_carbene', #these were ones that David's reactions were in (but not PFAS)
                 'CO_Disproportionation',
                 'Disproportionation', 
                 'H_Abstraction',
                 'R_Recombination',
                 'Singlet_Carbene_Intra_Disproportionation',  
                       ]

count=0
for fam in families_ive_rebased:
    
    if not os.path.exists(f'./all_families_on_rebase_edited_by_Nora/one_kinfam_at_a_time/{fam}_rebased/chemkin'):
        print(f'############## Starting family: {fam} #################')

        #reset the database
        clean_slate()

        #checkout out the new training data
        checkout_family(fam)

        #generate the RMG model
        generate_RMG_model(f'all_families_on_rebase_edited_by_Nora/one_kinfam_at_a_time/{fam}', input_type)
        count +=1
print(f'Count: {count}')

In [ ]:
#test other families outside of the ones I have rebased, maybe the issue has to do with a family that I didn't play around with, like Birad_recombination 

#making jobs for regenerating_model_at_different_commits/all_families_on_rebase_edited_by_Nora/one_kinfam_at_a_time/
input_type = 'PFAS_fams'

families_ive_rebased = ['CO_CF_bond_dissociation',
                 'PFAS_Hydrolysis', 
                 'Lactone_Formation',
                 'CO2_Elimination_From_Carboxylic_Acid',
                 'Perfluoroalkene_Formation',
                 'Enol_Ether_Formation',
    
                '1+2_Cycloaddition',
                 'XY_Addition_MultipleBond',
                 'R_Addition_MultipleBond',
                 '1,2_Insertion_CO', 
                 '1,3_Insertion_CO2',
                 '1,3_sigmatropic_rearrangement',
                 'R_Addition_COm',
                 'F_Abstraction',
                                
                 '1,2_Insertion_carbene', #these were ones that David's reactions were in (but not PFAS)
                 'CO_Disproportionation',
                 'Disproportionation', 
                 'H_Abstraction',
                 'R_Recombination',
                 'Singlet_Carbene_Intra_Disproportionation',  
                       ]

RMG_families_path = '/home/khalil.nor/Code/RMG-database/input/kinetics/families'
other_fams_that_I_havent_touched = [fam for fam in os.listdir(RMG_families_path) if fam not in families_ive_rebased and fam not in ['__pycache__','recommended.py']]


for fam in other_fams_that_I_havent_touched:
    
    if not os.path.exists(f'./all_families_on_rebase_edited_by_Nora/one_kinfam_at_a_time/{fam}_rebased/chemkin'):
        print(f'############## Starting family: {fam} #################')

        #reset the database
        clean_slate()

        #checkout out the new training data
        checkout_family(fam)

        #generate the RMG model
        generate_RMG_model(f'all_families_on_rebase_edited_by_Nora/one_kinfam_at_a_time/{fam}', input_type)

In [ ]:
#test other families outside of the ones I have rebased, maybe the issue has to do with a family that I didn't play around with, like Birad_recombination 

#making jobs for regenerating_model_at_different_commits/all_families_on_rebase_edited_by_Nora/one_kinfam_at_a_time/
input_type = 'PFAS_fams'

families_ive_rebased = ['CO_CF_bond_dissociation',
                 'PFAS_Hydrolysis', 
                 'Lactone_Formation',
                 'CO2_Elimination_From_Carboxylic_Acid',
                 'Perfluoroalkene_Formation',
                 'Enol_Ether_Formation',
    
                '1+2_Cycloaddition',
                 'XY_Addition_MultipleBond',
                 'R_Addition_MultipleBond',
                 '1,2_Insertion_CO', 
                 '1,3_Insertion_CO2',
                 '1,3_sigmatropic_rearrangement',
                 'R_Addition_COm',
                 'F_Abstraction',
                                
                 '1,2_Insertion_carbene', #these were ones that David's reactions were in (but not PFAS)
                 'CO_Disproportionation',
                 'Disproportionation', 
                 'H_Abstraction',
                 'R_Recombination',
                 'Singlet_Carbene_Intra_Disproportionation',  
                       ]

RMG_families_path = '/home/khalil.nor/Code/RMG-database/input/kinetics/families'
other_fams_that_I_havent_touched = [fam for fam in os.listdir(RMG_families_path) if fam not in families_ive_rebased and fam not in ['__pycache__','recommended.py']]


for fam in other_fams_that_I_havent_touched:
    
    if not os.path.exists(f'./all_families_on_rebase_edited_by_Nora/one_kinfam_at_a_time/{fam}_rebased/chemkin'):
        print(f'############## Starting family: {fam} #################')

        #reset the database
        clean_slate()

        #checkout out the new training data
        checkout_family(fam)

        #generate the RMG model
        generate_RMG_model(f'all_families_on_rebase_edited_by_Nora/one_kinfam_at_a_time/{fam}', input_type)

In [ ]:
#/regenerating_model_at_different_commits/all_families_involved_in_triplet_reaction

#reset the database
clean_slate()

In [ ]:
pdep_rxn_families = ["Birad_R_recombination",

"Birad_Recombination",

"R_Addition_MultipleBond",

"1+2_Cycloaddition",

"R_Recombination",

"Intra_Disproportionation",

"Intra_H_migration",

"1,2_Insertion_CO",

"1,3_Insertion_CO2"
                    ]

families_ive_rebased = ['CO_CF_bond_dissociation',
                 'PFAS_Hydrolysis', 
                 'Lactone_Formation',
                 'CO2_Elimination_From_Carboxylic_Acid',
                 'Perfluoroalkene_Formation',
                 'Enol_Ether_Formation',
    
                '1+2_Cycloaddition',
                 'XY_Addition_MultipleBond',
                 'R_Addition_MultipleBond',
                 '1,2_Insertion_CO', 
                 '1,3_Insertion_CO2',
                 '1,3_sigmatropic_rearrangement',
                 'R_Addition_COm',
                 'F_Abstraction',
                                
                 '1,2_Insertion_carbene', #these were ones that David's reactions were in (but not PFAS)
                 'CO_Disproportionation',
                 'Disproportionation', 
                 'H_Abstraction',
                 'R_Recombination',
                 'Singlet_Carbene_Intra_Disproportionation',  
                       ]

checkout_multiple_families(pdep_rxn_families)

#the only difference between the commit on rebased_PFAS_libs branch eb6010936b0d0b5b6ea4445db2238643bde851d5 and 908a25d5f354e34ba46d44c1bd47b76cad14b5fc from RMG-database branch PFAS_kinetics_rebased_Jan2025
#is : 

# modified:   input/kinetics/families/1+2_Cycloaddition/groups.py
# modified:   input/kinetics/families/1+2_Cycloaddition/rules.py
# modified:   input/kinetics/families/1+2_Cycloaddition/training/dictionary.txt
# modified:   input/kinetics/families/1+2_Cycloaddition/training/reactions.py
# modified:   input/kinetics/families/1,2_Insertion_CO/groups.py
# modified:   input/kinetics/families/1,2_Insertion_CO/rules.py
# modified:   input/kinetics/families/1,2_Insertion_CO/training/dictionary.txt
# modified:   input/kinetics/families/1,2_Insertion_CO/training/reactions.py
# modified:   input/kinetics/families/1,3_Insertion_CO2/groups.py
# modified:   input/kinetics/families/1,3_Insertion_CO2/rules.py
# modified:   input/kinetics/families/1,3_Insertion_CO2/training/dictionary.txt
# modified:   input/kinetics/families/1,3_Insertion_CO2/training/reactions.py
# modified:   input/kinetics/families/R_Addition_MultipleBond/groups.py
# modified:   input/kinetics/families/R_Addition_MultipleBond/rules.py
# modified:   input/kinetics/families/R_Addition_MultipleBond/training/dictionary.txt
# modified:   input/kinetics/families/R_Addition_MultipleBond/training/reactions.py
# modified:   input/kinetics/families/R_Recombination/groups.py
# modified:   input/kinetics/families/R_Recombination/rules.py
# modified:   input/kinetics/families/R_Recombination/training/dictionary.txt
# modified:   input/kinetics/families/R_Recombination/training/reactions.py


In [ ]:
#regenerating_model_at_different_commits/checking_out_families_from_before_rebase/git_checkout_one_family_at_a_time_plus_all_ive_rebased

input_type = 'PFAS_fams'

families_ive_rebased = ['CO_CF_bond_dissociation',
                 'PFAS_Hydrolysis', 
                 'Lactone_Formation',
                 'CO2_Elimination_From_Carboxylic_Acid',
                 'Perfluoroalkene_Formation',
                 'Enol_Ether_Formation',
    
                '1+2_Cycloaddition',
                 'XY_Addition_MultipleBond',
                 'R_Addition_MultipleBond',
                 '1,2_Insertion_CO', 
                 '1,3_Insertion_CO2',
                 '1,3_sigmatropic_rearrangement',
                 'R_Addition_COm',
                 'F_Abstraction',
                                
                 '1,2_Insertion_carbene', #these were ones that David's reactions were in (but not PFAS)
                 'CO_Disproportionation',
                 'Disproportionation', 
                 'H_Abstraction',
                 'R_Recombination',
                 'Singlet_Carbene_Intra_Disproportionation',  
                       ]

RMG_families_path = '/home/khalil.nor/Code/RMG-database/input/kinetics/families'
other_fams_that_I_havent_touched = [fam for fam in os.listdir(RMG_families_path) if fam not in families_ive_rebased and fam not in ['__pycache__','recommended.py']]


for fam in other_fams_that_I_havent_touched:
    
        print(f'############## Starting family: {fam} #################')

        #reset the database
        clean_slate()
        
        #checkout all of the families that I've rebased

        #checkout out the new training data
        checkout_family_from_before_rebase(fam)

        #generate the RMG model
        generate_RMG_model(f'all_families_on_rebase_edited_by_Nora/one_kinfam_at_a_time/{fam}', input_type)

In [ ]:
checkout_multiple_families_from_before_rebase(['1,2_Insertion_carbene', #these were ones that David's reactions were in (but not PFAS)
                 'CO_Disproportionation',
                 'Disproportionation', 
                 'H_Abstraction',
                 'R_Recombination',
                 'Singlet_Carbene_Intra_Disproportionation',  
                       ])

In [ ]:
#regenerating_model_at_different_commits/checking_out_families_from_before_rebase/git_checkout_one_family_at_a_time_plus_all_ive_rebased

input_type = 'PFAS_fams'

families_ive_rebased = ['CO_CF_bond_dissociation',
                 'PFAS_Hydrolysis', 
                 'Lactone_Formation',
                 'CO2_Elimination_From_Carboxylic_Acid',
                 'Perfluoroalkene_Formation',
                 'Enol_Ether_Formation',
    
                '1+2_Cycloaddition',
                 'XY_Addition_MultipleBond',
                 'R_Addition_MultipleBond',
                 '1,2_Insertion_CO', 
                 '1,3_Insertion_CO2',
                 '1,3_sigmatropic_rearrangement',
                 'R_Addition_COm',
                 'F_Abstraction',
                                
                 '1,2_Insertion_carbene', #these were ones that David's reactions were in (but not PFAS)
                 'CO_Disproportionation',
                 'Disproportionation', 
                 'H_Abstraction',
                 'R_Recombination',
                 'Singlet_Carbene_Intra_Disproportionation',  
                 'halocarbene_CO_dimerization'   #need to add in because halogens uses it
               ]



RMG_families_path = '/home/khalil.nor/Code/RMG-database/input/kinetics/families'
other_fams_that_I_havent_touched = [fam for fam in os.listdir(RMG_families_path) if fam not in families_ive_rebased and fam not in ['__pycache__','recommended.py']]


for fam in other_fams_that_I_havent_touched:
    
        if fam == '1,2-Birad_to_alkene':
            print('skipping 1,2-Birad_to_alkene')
            continue
    
        print(f'############## Starting family: {fam} #################')

        #reset the database
        clean_slate()
        
        #make sure to git checkout the recommended.py
        process = subprocess.run(['git', 'checkout', '908a25d5f354e34ba46d44c1bd47b76cad14b5fc', 'input/kinetics/families/recommended.py'], stdout=PIPE, stderr=PIPE, cwd='/home/khalil.nor/Code/RMG-database')
        print(process.stdout) 
        print(process.stderr)
        
        #checkout all of the families that I've rebased
        checkout_multiple_families_from_before_rebase(families_ive_rebased)

        #checkout out the new training data
        checkout_family_from_before_rebase(fam)

        #generate the RMG model
        generate_RMG_model(f'checking_out_families_from_before_rebase/git_checkout_one_family_at_a_time_plus_all_ive_rebased/{fam}', input_type)
        
        #wait until a chemkin file is generated 
        minutes = 4
        time.sleep(minutes*60)
        if os.path.exists(f'./checking_out_families_from_before_rebase/git_checkout_one_family_at_a_time_plus_all_ive_rebased/{fam}_rebased/chemkin'):
            print('Job has started!')
            continue
        else: 
            print('Have to wait a little longer')
            time.sleep(minutes*60) 
            if os.path.exists(f'./checking_out_families_from_before_rebase/git_checkout_one_family_at_a_time_plus_all_ive_rebased/{fam}_rebased/chemkin'):
                print('Ok lets go!')
                continue
            else: 
                print('Theres an issue...')
                break

        
        

skipping 1,2-Birad_to_alkene
############## Starting family: 1,2_Elimination_LiR #################
Restoring database to main.
b'HEAD is now at eb60109 Adding in new thermo library, from Caroline, for PFLactone carbene ethers (the species that generated charge separated resonance structures in 1+2_Cycloadditon)\n'
b''
b''
b''
Checking out the following families: ['CO_CF_bond_dissociation', 'PFAS_Hydrolysis', 'Lactone_Formation', 'CO2_Elimination_From_Carboxylic_Acid', 'Perfluoroalkene_Formation', 'Enol_Ether_Formation', '1+2_Cycloaddition', 'XY_Addition_MultipleBond', 'R_Addition_MultipleBond', '1,2_Insertion_CO', '1,3_Insertion_CO2', '1,3_sigmatropic_rearrangement', 'R_Addition_COm', 'F_Abstraction', '1,2_Insertion_carbene', 'CO_Disproportionation', 'Disproportionation', 'H_Abstraction', 'R_Recombination', 'Singlet_Carbene_Intra_Disproportionation', 'halocarbene_CO_dimerization'].
b''
b''
Checking out the following family: 1,2_Elimination_LiR.
b''
b"error: pathspec 'input/kinetics/fam

In [ ]:
other_fams_that_I_havent_touched

In [ ]:
clean_slate()

In [ ]:
checkout_multiple_families_from_before_rebase(families_ive_rebased)

# Scancel Jobs 

In [ ]:
    
jobs = [
          "48699967     short   S_CH3F khalil.n  R    1:25:12      1 c0205",
          "48699968     short   S_CH3F khalil.n  R    1:25:12      1 c0216",
          "48699969     short   S_CH3F khalil.n  R    1:25:12      1 c0227",
          "48699970     short   S_CH3F khalil.n  R    1:25:12      1 c0241",
          "48699971     short   S_CH3F khalil.n  R    1:25:12      1 c0248",
          "48699972     short   S_CH3F khalil.n  R    1:25:12      1 c0319",
          "48699973     short   S_CH3F khalil.n  R    1:25:12      1 c0324",
          "48699974     short   S_CH3F khalil.n  R    1:25:12      1 c2201",
          "48699976     short   S_CH3F khalil.n  R    1:25:12      1 c0208",
          "48699977     short   S_CH3F khalil.n  R    1:25:12      1 c0209",
          "48699978     short   S_CH3F khalil.n  R    1:25:12      1 c0234",
          "48699979     short   S_CH3F khalil.n  R    1:25:12      1 c0235",
          "48699980     short   S_CH3F khalil.n  R    1:25:12      1 c0222",
          "48699981     short   S_CH3F khalil.n  R    1:25:12      1 c0223",
          "48699982     short   S_CH3F khalil.n  R    1:25:12      1 c0225",
          "48699983     short   S_CH3F khalil.n  R    1:25:12      1 c0237",
          "48699984     short   S_CH3F khalil.n  R    1:25:12      1 c0238",
          "48699985     short   S_CH3F khalil.n  R    1:25:12      1 c0239",
          "48699986     short   S_CH3F khalil.n  R    1:25:12      1 c0243",
          "48699987     short   S_CH3F khalil.n  R    1:25:12      1 c0244",
          "48699988     short   S_CH3F khalil.n  R    1:25:12      1 c0245",

]

job_ids = []
for line in jobs: 
    match = re.search('^([0-9]+)     short   S_CH3F', line).group(1)
    print(match)
    job_ids.append(match)


In [ ]:
#copy the necessary files to the folder
for job_id in job_ids: 
    process = subprocess.run(['scancel', job_id], stdout=PIPE, stderr=PIPE, encoding='utf-8')
    print(process.stdout)
    print(process.stderr)

# Testing Flamespeeds

In [19]:
#testing CH3F main

loglevel=0
chemkin_indices=[124]
description = 'CH3F_main/CH3F_main'
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

*******************Starting cti: copy_chem0124.cti of CH3F_main/CH3F_main ***************
****************************starting new volume fraction: 0.125 **************************
********************passed volume fraction:0.125, error: [Errno 2] No such file or directory: 'data/0.125.csv'*************************************
flamespeed: [0.15710163310186137]


In [20]:
print('hi')

hi


In [ ]:
#testing CH3F main

loglevel=0
chemkin_indices=[60]
description = 'CH3F_main'
convert_to_cti(description, chemkin_indices)
test_flamespeed(description, chemkin_indices, loglevel)

In [ ]:
#testing CH3F main

loglevel=0
chemkin_indices=[12, 30]
description = 'CH3F_main/CH3F_main'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [ ]:
#testing CH3F main with ANL_Brown_pdep
loglevel=0
chemkin_indices=[15,30]
description = 'CH3F_main/CH3F_main_with_ANL_Brown_pdep'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [ ]:
#testing CH3F main with ANL_Brown_pdep
loglevel=0
chemkin_indices=[60]
description = 'CH3F_main/CH3F_main_with_ANL_Brown_pdep'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [21]:
#testing CH3F main with ANL_Brown_pdep
loglevel=0
chemkin_indices=[101]
description = 'CH3F_main/CH3F_main_with_ANL_Brown_pdep'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

b''
b''
/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/CH3F_main/CH3F_main_with_ANL_Brown_pdep/
b"Running Cantera Version: 2.6.0\n*******************************needs some work****************************************\nUnmarked duplicates on lines 4885 and 9698\nEditing chemkin file to allow conversion to .cti\nreading in files\nHF(39)+CF2O(48)+CH2CO(27)(+M)<=>O[C](F)F(3156)+C2H2FO(458)(+M) 1.000e+00 0.000     0.000    \n HF(39)+CF2O(48)+CH2CO(27)(+M)<=>O[C](F)F(3156)+C2H2FO(458)(+M) 1.000e+00 0.000     0.000    \n\nno match\n4884\nHF(39)+CF2O(48)+CH2CO(27)(+M)<=>O[C](F)F(3156)+C2H2FO(458)(+M) 1.000e+00 0.000     0.000    \n\nno match\n4885\n    TCHEB/ 300.000   2500.000 /\n\nno match\n4886\n    PCHEB/ 0.010     98.692   /\n\nno match\n4887\n    CHEB/ 6 4/\n\nno match\n4888\n    CHEB/ -1.588e+01   -1.547e-04   -1.077e-04   -5.978e-05  /\n\nno match\n4889\n   

In [ ]:
#all_families_on_rebase_edited_by_Nora

loglevel=0
chemkin_indices=[40]
description = 'all_families_on_rebase_edited_by_Nora'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [ ]:
#all_families_on_rebase_edited_by_Nora

loglevel=0
chemkin_indices=[54]
description = 'all_families_on_rebase_edited_by_Nora'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

## We are seeing the problem come up here

In [ ]:
#all_families_on_rebase_edited_by_Nora/MODEL_with_newCH2F2_PFAS_libs/
loglevel=0
description = 'all_families_on_rebase_edited_by_Nora/MODEL_with_newCH2F2_PFAS_libs'
chemkin_indices = [18]
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [ ]:
path_ = '/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/all_families_on_rebase_edited_by_Nora/one_kinlib_at_a_time/'
descriptions = [folder for folder in os.listdir(path_) if 'exclude' in folder and os.path.isdir(path_+folder)]

chemkin_indices = [18]
for desc in descriptions:
    description = f'all_families_on_rebase_edited_by_Nora/one_kinlib_at_a_time/{desc}'
    convert_to_cti(description, chemkin_indices, format_desc=False)
    test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)


In [ ]:
#all_families_on_rebase_edited_by_Nora/without_CH2F2_or_PFAS_kinetic_libs/
loglevel=0
description = 'all_families_on_rebase_edited_by_Nora/without_CH2F2_or_PFAS_kinetic_libs'
chemkin_indices = [70]
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [ ]:
#all_families_changed_by_Nora_and_new_PFAS

loglevel=0
chemkin_indices=[20]
description = 'all_families_changed_by_Nora_and_new_PFAS'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [ ]:
#all_families_changed_by_Nora_and_new_PFAS

loglevel=0
chemkin_indices=[27]
description = 'all_families_changed_by_Nora_and_new_PFAS'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [ ]:
loglevel=0
chemkin_indices=[37]
description = 'all_families_on_rebase_edited_by_Nora/MODEL_with_newCH2F2_PFAS_libs'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [ ]:
loglevel=0
chemkin_indices=[34]
description = 'checking_out_families_from_before_rebase/families_that_I_rebased_plus_davids/'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [ ]:
#rebased_RMG-Py-database/regenerating_model_at_different_commits/all_families_on_rebase_edited_by_Nora/MODEL_with_newCH2F2_PFAS_libs/all_intended_families_included/

loglevel=0
chemkin_indices=[44]
description = 'all_families_on_rebase_edited_by_Nora/MODEL_with_newCH2F2_PFAS_libs/all_intended_families_included'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [ ]:
#rebased_RMG-Py-database/regenerating_model_at_different_commits/all_families_on_rebase_edited_by_Nora/MODEL_with_newCH2F2_PFAS_libs/all_intended_families_included_PFAS_fams/

loglevel=0
chemkin_indices=[106]
description = 'all_families_on_rebase_edited_by_Nora/MODEL_with_newCH2F2_PFAS_libs/all_intended_families_included_PFAS_fams'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [ ]:
#rebased_RMG-Py-database/regenerating_model_at_different_commits/all_families_on_rebase_edited_by_Nora/MODEL_with_newCH2F2_PFAS_libs/all_intended_families_included_PFAS_fams_triplet_lib/

loglevel=0
chemkin_indices=[125]
description = 'all_families_on_rebase_edited_by_Nora/MODEL_with_newCH2F2_PFAS_libs/all_intended_families_included_PFAS_fams_triplet_lib'
#convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [ ]:
path_='/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/CH3F_main/edits_to_CH3F_main_with_ABL/copy_chem0101_subbing_rxn.cti'

cti= 'subbing_cti'

cti_path = path_

print(f'*******************Starting cti: {cti} ***************')
gas = ct.Solution(cti_path)
halocarbon = 'CH3F(1)'

To = 298
Po = 1e5 # ct.one_atm


mole_frac_list = list(np.linspace(0.025, 0.25, 50))
mole_frac_list=[0.125]

results = {}

for i in range(len(mole_frac_list)): 
    try: 
        x = mole_frac_list[i]
        string = f'****************************starting new volume fraction: {x} **************************'
        print(string)

        norm_ox = (1-x)*.21
        mole_frac_dict = {halocarbon: x, 'O2(2)':((1-x)*.21), 'N2':((1-x)*0.79)} 
        #print(f'Unnormalized composition dictionary: {mole_frac_dict}')

        #normalize it to O2 
        mole_frac_dict = {halocarbon: (x/norm_ox), 'O2(2)':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox } 
        #print(f'Normalizing: {mole_frac_dict}')

        gas.TPX = To, Po, mole_frac_dict
        width = 0.08
        flame = ct.FreeFlame(gas, width=width)
        flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1) 
        flame.max_time_step_count = 3000
        flame.solve(loglevel=loglevel, auto=False)
        Su = flame.velocity[0]
        results[x] = Su
        if save==True: 
            sltn = flame.to_solution_array()
            pd = sltn.to_pandas()
            pd.to_csv(f'data/{x}.csv', index=False)

    except Exception as e: 
        print(f'********************passed volume fraction:{mole_frac_list[i]}, error: {e}*************************************')
        pass

vol_fracs = list(results.keys())
flame_speeds = list(results.values())

print(f"flamespeed: {flame_speeds}")


#this is the CH3F_main_with_ABL, with one subbed reaction. This tests to see if subbing out that one reaction : O2(2) + CH2(T)(17) (+ M) <=> H(3) + CHO2(76) (+ M)
#would give us a reasonable flamespeed

#substituting this pdep rxn for what I had on 

In [ ]:
#all_families_Ive_ever_touched WITH library for triplet reaction
#projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/all_families_I_have_ever_touched/chemkin/chem_edge.inp
loglevel=0
chemkin_indices=[15,25]
description = 'all_families_I_have_ever_touched'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [ ]:
#all_families_Ive_ever_touched WITH library for triplet reaction
#projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/all_families_I_have_ever_touched/chemkin/chem_edge.inp
loglevel=0
chemkin_indices=[38]
description = 'all_families_I_have_ever_touched'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [ ]:
#all_families_Ive_ever_touched WITH library for triplet reaction
#projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/all_families_I_have_ever_touched/chemkin/chem_edge.inp
loglevel=0
chemkin_indices=[69]
description = 'all_families_I_have_ever_touched'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [ ]:
loglevel=0
chemkin_indices=[21]
for fam in families_ive_rebased:
    description = f'all_families_on_rebase_edited_by_Nora/one_kinfam_at_a_time/{fam}_rebased'
    convert_to_cti(description, chemkin_indices, format_desc=False)
    test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [ ]:
families_ive_rebased = ['CO_CF_bond_dissociation',
                 'PFAS_Hydrolysis', 
                 'Lactone_Formation',
                 'CO2_Elimination_From_Carboxylic_Acid',
                 'Perfluoroalkene_Formation',
                 'Enol_Ether_Formation',
    
                '1+2_Cycloaddition',
                 'XY_Addition_MultipleBond',
                 'R_Addition_MultipleBond',
                 '1,2_Insertion_CO', 
                 '1,3_Insertion_CO2',
                 '1,3_sigmatropic_rearrangement',
                 'R_Addition_COm',
                 'F_Abstraction',
                                
                 '1,2_Insertion_carbene', #these were ones that David's reactions were in (but not PFAS)
                 'CO_Disproportionation',
                 'Disproportionation', 
                 'H_Abstraction',
                 'R_Recombination',
                 'Singlet_Carbene_Intra_Disproportionation',  
                       ]

loglevel=0
chemkin_indices=[48]
for fam in families_ive_rebased:
    description = f'all_families_on_rebase_edited_by_Nora/one_kinfam_at_a_time/families_that_I_have_rebased/{fam}_rebased'
    convert_to_cti(description, chemkin_indices, format_desc=False)
    test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [ ]:
chemkin_indices = [29]
for fam in other_fams_that_I_havent_touched:

    description = f'all_families_on_rebase_edited_by_Nora/one_kinfam_at_a_time/{fam}_rebased'
    convert_to_cti(description, chemkin_indices, format_desc=False)
    test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

# #The current git HEAD for RMG-Py is:
# 	b'68b2eeeaa489dd2cf4b4bc1c8cf8b1e6a416d179'
# 	b'Tue Apr 8 16:05:01 2025 -0400'

# The current git HEAD for RMG-database is:
# 	b'4fee96d55dc156935f56a4cd4db68c7ab36cfe85'
	# b'Fri May 9 15:18:17 2025 -0400'
#

#This was mistakenly on the commit 4fee96d55dc156935f56a4cd4db68c7ab36cfe85 (HEAD -> rebased_Feb2025_and_including_PFLactone_carbene_ether_thermo) , 
# not on the clean main. 

In [ ]:
loglevel=0
chemkin_indices=[20]

description = f'all_families_involved_in_triplet_reaction'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [ ]:
loglevel=0
chemkin_indices=[36]

description = 'all_families_on_rebase_edited_by_Nora/MODEL_with_newCH2F2_PFAS_libs/all_intended_families_included_PFAS_fams_new_thermos'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

In [6]:
loglevel=0
chemkin_indices=[36]

description = 'all_families_on_rebase_edited_by_Nora/MODEL_with_newCH2F2_PFAS_libs/all_intended_families_included_PFAS_fams_before_rebase'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)




b''
b''
/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/all_families_on_rebase_edited_by_Nora/MODEL_with_newCH2F2_PFAS_libs/all_intended_families_included_PFAS_fams_before_rebase/
b'Running Cantera Version: 2.6.0\n36\n**************************command passed, converting to cti***************************\n'
b'sh: line 0: source: activate: file not found\n'
finished
*******************Starting cti: copy_chem0036.cti of all_families_on_rebase_edited_by_Nora/MODEL_with_newCH2F2_PFAS_libs/all_intended_families_included_PFAS_fams_before_rebase ***************
****************************starting new volume fraction: 0.125 **************************
********************passed volume fraction:0.125, error: [Errno 2] No such file or directory: 'data/0.125.csv'*************************************
flamespeed: [0.6027840081057806]


In [7]:
loglevel=0
chemkin_indices=[48]

description = 'checking_out_families_from_before_rebase/families_that_I_rebased_plus_R_Add_from_before'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)



b''
b''
/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/checking_out_families_from_before_rebase/families_that_I_rebased_plus_R_Add_from_before/
b"Running Cantera Version: 2.6.0\n48\n*******************************needs some work****************************************\nUndeclared duplicate reactions error. See: WARNING:root:The CTI input file format is deprecated and will be removed in Cantera 3.0.\nUse 'ck2yaml.py' to convert Chemkin-format input files to the YAML format.\nSee https://cantera.org/tutorials/ck2yaml-tutorial.html for more information.\n/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/cantera/ck2cti.py:2358: DeprecationWarning: XML_Node::build: \nThe CTI and XML input file formats are deprecated and will be removed in\nCantera 3.0. Use 'cti2yaml.py' or 'ctml2yaml.py' to convert CTI or XML input\nfiles to the YAML format.

In [8]:
loglevel=0
chemkin_indices=[20]
description = 'checking_out_families_from_before_rebase/birad_recombination_from_before'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)


b''
b''
/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/checking_out_families_from_before_rebase/birad_recombination_from_before/
b'Running Cantera Version: 2.6.0\n20\n**************************command passed, converting to cti***************************\n'
b'sh: line 0: source: activate: file not found\n'
finished
*******************Starting cti: copy_chem0020.cti of checking_out_families_from_before_rebase/birad_recombination_from_before ***************
****************************starting new volume fraction: 0.125 **************************
********************passed volume fraction:0.125, error: [Errno 2] No such file or directory: 'data/0.125.csv'*************************************
flamespeed: [2.009406146232853]


In [6]:

loglevel=0
chemkin_indices=[86]
families_in_here = [re.search('(\S+)_rebased', fam).group(1) for fam in os.listdir('checking_out_families_from_before_rebase/checkout_Birad_recomb_plus_one_other_family/') if 'rebased' in fam]
for fam in families_in_here:
    try: 
        description = f'checking_out_families_from_before_rebase/checkout_Birad_recomb_plus_one_other_family/{fam}'
        #convert_to_cti(description, chemkin_indices)
        test_flamespeed(description, chemkin_indices, loglevel)
    except: 
        pass


*******************Starting cti: copy_chem0086.cti of checking_out_families_from_before_rebase/checkout_Birad_recomb_plus_one_other_family/1+2_Cycloaddition ***************
****************************starting new volume fraction: 0.125 **************************
********************passed volume fraction:0.125, error: [Errno 2] No such file or directory: 'data/0.125.csv'*************************************
flamespeed: [0.5391190564622598]
*******************Starting cti: copy_chem0086.cti of checking_out_families_from_before_rebase/checkout_Birad_recomb_plus_one_other_family/1,2_Insertion_CO ***************
****************************starting new volume fraction: 0.125 **************************
********************passed volume fraction:0.125, error: [Errno 2] No such file or directory: 'data/0.125.csv'*************************************
flamespeed: [0.5393062913627479]
*******************Starting cti: copy_chem0086.cti of checking_out_families_from_before_rebase/checkout_Birad_r

In [16]:
loglevel=0
chemkin_indices=[46]

description = 'checking_out_families_from_before_rebase/git_checkout_one_family_at_a_time_plus_all_ive_rebased/Birad_recombination_rebased'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

b''
b''
/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/checking_out_families_from_before_rebase/git_checkout_one_family_at_a_time_plus_all_ive_rebased/Birad_recombination_rebased//
b'Running Cantera Version: 2.6.0\n46\n**************************command passed, converting to cti***************************\n'
b'sh: line 0: source: activate: file not found\n'
finished
*******************Starting cti: copy_chem0046.cti of checking_out_families_from_before_rebase/git_checkout_one_family_at_a_time_plus_all_ive_rebased/Birad_recombination_rebased/ ***************
****************************starting new volume fraction: 0.125 **************************
********************passed volume fraction:0.125, error: [Errno 2] No such file or directory: 'data/0.125.csv'*************************************
flamespeed: [0.26485233969884936]


In [4]:
loglevel=0
chemkin_indices=[21]
families_in_here = [re.search('(\S+)_rebased', fam).group(1) for fam in os.listdir('checking_out_families_from_before_rebase/git_checkout_one_family_at_a_time_plus_all_ive_rebased/') if 'rebased' in fam]
for fam in families_in_here:
    try: 
        description = f'checking_out_families_from_before_rebase/git_checkout_one_family_at_a_time_plus_all_ive_rebased/{fam}'
        convert_to_cti(description, chemkin_indices)
        test_flamespeed(description, chemkin_indices, loglevel)
    except: 
        pass

b''
b''
/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/checking_out_families_from_before_rebase/git_checkout_one_family_at_a_time_plus_all_ive_rebased/1,2-Birad_to_alkene_rebased/
b'Running Cantera Version: 2.6.0\n21\n**************************command passed, converting to cti***************************\n'
b'sh: line 0: source: activate: file not found\n'
finished
*******************Starting cti: copy_chem0021.cti of checking_out_families_from_before_rebase/git_checkout_one_family_at_a_time_plus_all_ive_rebased/1,2-Birad_to_alkene ***************
****************************starting new volume fraction: 0.125 **************************


/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:157: DeprecationWarning: XML_Node::build: 
The CTI and XML input file formats are deprecated and will be removed in
Cantera 3.0. Use 'cti2yaml.py' or 'ctml2yaml.py' to convert CTI or XML input
files to the YAML format. See https://cantera.org/tutorials/legacy2yaml.html
for more information.


********************passed volume fraction:0.125, error: [Errno 2] No such file or directory: 'data/0.125.csv'*************************************
flamespeed: [0.8072108167444434]
b''
b''
/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/checking_out_families_from_before_rebase/git_checkout_one_family_at_a_time_plus_all_ive_rebased/1,2_Elimination_LiR_rebased/
b'Running Cantera Version: 2.6.0\n21\n**************************command passed, converting to cti***************************\n'
b'sh: line 0: source: activate: file not found\n'
finished
*******************Starting cti: copy_chem0021.cti of checking_out_families_from_before_rebase/git_checkout_one_family_at_a_time_plus_all_ive_rebased/1,2_Elimination_LiR ***************
****************************starting new volume fraction: 0.125 **************************
********************passed volume fraction:0.

In [15]:
#all_families_on_rebase_edited_by_Nora

loglevel=0
chemkin_indices=[102]
description = 'adding_pdep_rxns_to_ABLib'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

b''
b''
/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/adding_pdep_rxns_to_ABLib/
b"Running Cantera Version: 2.6.0\n*******************************needs some work****************************************\nUndeclared duplicate reactions error. See: WARNING:root:The CTI input file format is deprecated and will be removed in Cantera 3.0.\nUse 'ck2yaml.py' to convert Chemkin-format input files to the YAML format.\nSee https://cantera.org/tutorials/ck2yaml-tutorial.html for more information.\n/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/cantera/ck2cti.py:2358: DeprecationWarning: XML_Node::build: \nThe CTI and XML input file formats are deprecated and will be removed in\nCantera 3.0. Use 'cti2yaml.py' or 'ctml2yaml.py' to convert CTI or XML input\nfiles to the YAML format. See https://cantera.org/tutorials/legacy2yaml.html\nfor more info

In [13]:
#all_families_on_rebase_edited_by_Nora

loglevel=0
chemkin_indices=[51]
description = 'shutting_down_pathways_with_zero_rate'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)



b''
b''
/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/shutting_down_pathways_with_zero_rate/
b'Running Cantera Version: 2.6.0\n51\n**************************command passed, converting to cti***************************\n'
b'sh: line 0: source: activate: file not found\n'
finished
*******************Starting cti: copy_chem0051.cti of shutting_down_pathways_with_zero_rate ***************
****************************starting new volume fraction: 0.125 **************************
********************passed volume fraction:0.125, error: [Errno 2] No such file or directory: 'data/0.125.csv'*************************************
flamespeed: [0.013284512489232307]


In [11]:

loglevel=0
chemkin_indices=[131]
description = 'shutting_down_pathways_with_zero_rate_FFCM_seed'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

#the zero rates and the FFCM seed only give me as good as the regular CH3F main estimate. 

b''
b''
/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/shutting_down_pathways_with_zero_rate_FFCM_seed/
b'Running Cantera Version: 2.6.0\n**************************command passed, converting to cti***************************\n'
b'sh: line 0: source: activate: file not found\n'
finished
*******************Starting cti: copy_chem0131.cti of shutting_down_pathways_with_zero_rate_FFCM_seed ***************
****************************starting new volume fraction: 0.125 **************************
********************passed volume fraction:0.125, error: [Errno 2] No such file or directory: 'data/0.125.csv'*************************************
flamespeed: [0.15454576877381462]


In [20]:
#all_families_on_rebase_edited_by_Nora

loglevel=0
chemkin_indices=[103]
description = "shutting_down_pathways_forcing_singlet_FFCM_seed"
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

b''
b''
/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/shutting_down_pathways_forcing_singlet_FFCM_seed/
b'Running Cantera Version: 2.6.0\n**************************command passed, converting to cti***************************\n'
b'sh: line 0: source: activate: file not found\n'
finished
*******************Starting cti: copy_chem0103.cti of shutting_down_pathways_forcing_singlet_FFCM_seed ***************
****************************starting new volume fraction: 0.125 **************************
********************passed volume fraction:0.125, error: [Errno 2] No such file or directory: 'data/0.125.csv'*************************************
flamespeed: [0.15455823022000853]


In [12]:
loglevel=0
chemkin_indices=[127]
description = "species_in_core/adding_CHO2_singletO2_OOCF_OC1FOO1_FFCM_seed"
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

b''
b''
/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/species_in_core/adding_CHO2_singletO2_OOCF_OC1FOO1_FFCM_seed/
b'Running Cantera Version: 2.6.0\n**************************command passed, converting to cti***************************\n'
b'sh: line 0: source: activate: file not found\n'
finished
*******************Starting cti: copy_chem0127.cti of species_in_core/adding_CHO2_singletO2_OOCF_OC1FOO1_FFCM_seed ***************
****************************starting new volume fraction: 0.125 **************************
********************passed volume fraction:0.125, error: [Errno 2] No such file or directory: 'data/0.125.csv'*************************************
flamespeed: [0.154536091853795]


In [14]:
loglevel=0
chemkin_indices=[52]
description = 'added_pdep_rxns_to_ABLib_and_new_thermos'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

b''
b''
/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/added_pdep_rxns_to_ABLib_and_new_thermos/
b"Running Cantera Version: 2.6.0\n52\n*******************************needs some work****************************************\nUndeclared duplicate reactions error. See: WARNING:root:The CTI input file format is deprecated and will be removed in Cantera 3.0.\nUse 'ck2yaml.py' to convert Chemkin-format input files to the YAML format.\nSee https://cantera.org/tutorials/ck2yaml-tutorial.html for more information.\n/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/cantera/ck2cti.py:2358: DeprecationWarning: XML_Node::build: \nThe CTI and XML input file formats are deprecated and will be removed in\nCantera 3.0. Use 'cti2yaml.py' or 'ctml2yaml.py' to convert CTI or XML input\nfiles to the YAML format. See https://cantera.org/tutorials/legacy2yaml.

In [14]:
loglevel=0
chemkin_indices=[99]
description = 'species_in_core/adding_multiple_species_to_core'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

b''
b''
/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/species_in_core/adding_multiple_species_to_core/
b'Running Cantera Version: 2.6.0\n99\n**************************command passed, converting to cti***************************\n'
b'sh: line 0: source: activate: file not found\n'
finished
*******************Starting cti: copy_chem0099.cti of species_in_core/adding_multiple_species_to_core ***************
****************************starting new volume fraction: 0.125 **************************
********************passed volume fraction:0.125, error: [Errno 2] No such file or directory: 'data/0.125.csv'*************************************
flamespeed: [0.15454235971277394]


In [16]:
loglevel=0
chemkin_indices=[158]
description = 'species_in_core/adding_multiple_species_to_core'
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)

b''
b''
/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/species_in_core/adding_multiple_species_to_core/
b'Running Cantera Version: 2.6.0\n**************************command passed, converting to cti***************************\n'
b'sh: line 0: source: activate: file not found\n'
finished
*******************Starting cti: copy_chem0158.cti of species_in_core/adding_multiple_species_to_core ***************
****************************starting new volume fraction: 0.125 **************************
********************passed volume fraction:0.125, error: [Errno 2] No such file or directory: 'data/0.125.csv'*************************************
flamespeed: [0.1546018262347424]


In [5]:
loglevel=0
chemkin_indices=[96]
description = "FINAL_model"
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)


b''
b''
/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/FINAL_model/
b"Running Cantera Version: 2.6.0\n96\n*******************************needs some work****************************************\nUndeclared duplicate reactions error. See: WARNING:root:The CTI input file format is deprecated and will be removed in Cantera 3.0.\nUse 'ck2yaml.py' to convert Chemkin-format input files to the YAML format.\nSee https://cantera.org/tutorials/ck2yaml-tutorial.html for more information.\n/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/cantera/ck2cti.py:2358: DeprecationWarning: XML_Node::build: \nThe CTI and XML input file formats are deprecated and will be removed in\nCantera 3.0. Use 'cti2yaml.py' or 'ctml2yaml.py' to convert CTI or XML input\nfiles to the YAML format. See https://cantera.org/tutorials/legacy2yaml.html\nfor more information.\n

In [7]:
loglevel=0
chemkin_indices=[25]
description = "FINAL_model"
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)


b''
b''
/work/westgroup/nora/Code/projects/halogens/refrigerants/halogens_paper/models_diff/CH3F_before_and_after_rebase/rebased_RMG-Py-database/regenerating_model_at_different_commits/FINAL_model/
b'Running Cantera Version: 2.6.0\n25\n**************************command passed, converting to cti***************************\n'
b'sh: line 0: source: activate: file not found\n'
finished
*******************Starting cti: copy_chem0025.cti of FINAL_model ***************
****************************starting new volume fraction: 0.125 **************************
********************passed volume fraction:0.125, error: [Errno 2] No such file or directory: 'data/0.125.csv'*************************************
flamespeed: [0.2996901054016134]


In [ ]:
description = 'species_in_core/shutting_down_pathways_with_zero_rate'
loglevel=0
chemkin_indices=[25]
convert_to_cti(description, chemkin_indices, format_desc=False)
test_flamespeed(description, chemkin_indices, loglevel, format_desc=False)


# Playing around with F Abstraction

In [ ]:
loglevel=0
for fam in halocarbon_fams:
    if fam=="F_Abstraction": 
        if os.path.exists(f'./{fam}_rebased/chemkin'):
            print(f'############## Starting family: {fam} #################')
            chemkin_indices = [80,100]
            convert_to_cti(fam, chemkin_indices)
            test_flamespeed(fam, chemkin_indices, loglevel)

In [ ]:
loglevel=0
for fam in halocarbon_fams:
    if fam=="F_Abstraction": 
        if os.path.exists(f'./{fam}_rebased/chemkin'):
            print(f'############## Starting family: {fam} #################')
            chemkin_indices = [80,100]
            #convert_to_cti(fam, chemkin_indices)
            test_flamespeed(fam, chemkin_indices, loglevel)

In [ ]:
loglevel=0
for fam in halocarbon_fams:
    if fam=="F_Abstraction": 
        if os.path.exists(f'./{fam}_rebased/chemkin'):
            print(f'############## Starting family: {fam} #################')
            chemkin_indices = [30]
            convert_to_cti(fam, chemkin_indices)
            test_flamespeed(fam, chemkin_indices, loglevel)

# Speeds for multiple families

In [ ]:
loglevel=0
for fam in halocarbon_fams:
    if f'{fam}_speed.csv' not in os.listdir('./calculations'): 
        if os.path.exists(f'./{fam}_rebased/chemkin'):
            print(f'############## Starting family: {fam} #################')
            chemkin_indices = [20]
            convert_to_cti(fam, chemkin_indices)
            test_flamespeed(fam, chemkin_indices, loglevel, save=False)

In [ ]:
#review saved .csvs for flamespeeds 

csvs = [csv for csv in os.listdir('./calculations') if '.csv' in csv]

for csv in csvs: 
    df = pd.read_csv(f'./calculations/{csv}')
    print(csv, df['velocity'][0])

I don't think the issue is the rebased families separately? No issues with flamespeeds are coming up when I checkout families one at a time. 